In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import holodeck2 as holo
from holodeck2 import physics, utils

In [ ]:
sam = holo.sam.SAM()

In [ ]:
edges_3d, ndens_3d = sam.number_density_3d()

In [ ]:
fobs_gw_cents, fobs_gw_edges = physics.pta_freqs()
cents_4d, numb_4d = sam.number_total_4d_gwonly_instant(fobs_gw_edges, edges_3d, ndens_3d)

In [ ]:
fig, axes = plt.subplots(figsize=[12, 5], ncols=3)
labels = ['m1', 'm2', 'z']
ymax = np.sum(ndens_3d)
ymin = ymax / 1e10

for ii, ax in enumerate(axes):
    xx = edges_3d[ii]
    ax.set(
        xscale='log', xlabel=labels[ii],
        yscale='log', ylabel='Density', ylim=[ymin, ymax],
    )

    margin = [0, 1, 2]
    margin.pop(ii)
    yy = np.sum(ndens_3d, axis=tuple(margin))

    ax.plot(xx, yy)

plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=[12, 10], ncols=3, nrows=3)
labels = ['m1', 'm2', 'z']
ymax = np.sum(ndens_3d)
ymin = ymax / 1e10

for (ii, jj), ax in np.ndenumerate(axes):
    if ii < jj:
        ax.set_visible(False)
        continue

    ax.set(
        xscale='log',
        # xlabel=labels[ii],
        yscale='log',
        # ylim=[ymin, ymax],
        # ylabel='Density',
    )


    xx = edges_3d[jj]

    # ---- 1D
    if ii == jj:
        margin = [0, 1, 2]
        margin.pop(jj)
        yy = np.sum(ndens_3d, axis=tuple(margin))

        ax.plot(xx, yy)
        ax.set(ylim=[ymin, ymax])

    # ---- 2D
    else:
        yy = edges_3d[ii]
        mesh = np.meshgrid(xx, yy, indexing='ij')

        margin = [0, 1, 2]
        margin.pop(ii)  # ii > jj, so do ii first
        margin.pop(jj)
        zz = np.sum(ndens_3d, axis=tuple(margin))

        ax.pcolormesh(*mesh, np.log10(zz), shading='gouraud')



plt.show()